In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import xarray as xr
from matplotlib import pyplot as plt
import pandas as pd
import sys
sys.path.append('/home/onno/Thesis/Scripts')
import my_tools
from my_tools import plot_dic, file_dic
%matplotlib qt
import matplotlib.dates as mdates

In [2]:
path = '/media/onno/Algemeen/Thesis/GFS_T850/'

In [3]:
df_MAE_GFS = pd.read_csv(path+'MAE/Mean_Absolute_Error_lead_day_5_lon_6_14_lat_54_46_GFS.csv',index_col=0)
df_MAE_GFS.index = pd.to_datetime(df_MAE_GFS.index)
df_MAE_ERA5RF = pd.read_csv(path+'MAE/Mean_Absolute_Error_lead_day_5_lon_6_14_lat_54_46_ERA5RF.csv',index_col=0)
df_MAE_ERA5RF.index = pd.to_datetime(df_MAE_ERA5RF.index)
df_GSS_GFS = pd.read_csv(path+'GSS/Gilbert_Skill_Score_lead_day_5_lon_6_14_lat_54_46_GFS.csv',index_col=0)
df_GSS_GFS.index = pd.to_datetime(df_GSS_GFS.index)
df_GSS_ERA5RF = pd.read_csv(path+'GSS/Gilbert_Skill_Score_lead_day_5_lon_6_14_lat_54_46_ERA5RF.csv',index_col=0)
df_GSS_ERA5RF.index = pd.to_datetime(df_GSS_ERA5RF.index)





Best and worst annual forecasts

In [10]:
path_to_write = '/media/onno/Algemeen/Thesis/GFS_T850/rank_forecast/'
lon_0,lon_1,lat_0,lat_1 = (6,14,54,46)
#Good forecasts
columnz = ['persistent_hw','persistent_cw']
for column in columnz:
    
    column_MAE = column+'_MAE'
    df_good = df_MAE_ERA5RF.loc[:,[column]].dropna().rename(columns={column:column_MAE})
    df_good[column+'_GSS']=df_GSS_ERA5RF.loc[:,[column]].dropna()
    # df_MAE['rank'] = df_MAE[column_MAE].rank(method='min')
    df_good = df_good.sort_values(column+'_MAE').where(df_good[column+'_GSS']>0.6).dropna()
    df_good[:10].to_csv(path_to_write+'good_forecasts_{}_lon_{}_{}_lat_{}_{}.txt'.format(column,lon_0,lon_1,lat_0,lat_1))

    #Bad forecasts
    column_MAE = column+'_MAE'
    df_bad = df_MAE_ERA5RF.loc[:,[column]].dropna().rename(columns={column:column_MAE})
    df_bad[column+'_GSS']=df_GSS_ERA5RF.loc[:,[column]].dropna()
    # df_MAE['rank'] = df_MAE[column_MAE].rank(method='min')
    df_bad = df_bad.sort_values(column+'_MAE',ascending=False).where(df_bad[column+'_GSS']<0.2).dropna()
    df_bad[:10].to_csv(path_to_write+'bad_forecasts_{}_lon_{}_{}_lat_{}_{}.txt'.format(column,lon_0,lon_1,lat_0,lat_1))

Best and worst seasonal forecasts

In [20]:
path_to_write = '/media/onno/Algemeen/Thesis/GFS_T850/rank_forecast/seasonal/'
columnz = ['persistent_hw','persistent_cw']

seasonz = {'MAM':[3,4,5],
          'JJA':[6,7,8],
          'SON':[9,10,11],
          'DJF':[12,1,2]}
for column in columnz:
    for season in seasonz:
    
        column_MAE = column+'_MAE'
        df_good = df_MAE_ERA5RF.loc[:,[column]].dropna().rename(columns={column:column_MAE})
        df_good[column+'_GSS']=df_GSS_ERA5RF.loc[:,[column]].dropna()
        # df_MAE['rank'] = df_MAE[column_MAE].rank(method='min')
        df_good = df_good.sort_values(column+'_MAE').where(df_good[column+'_GSS']>0.6).dropna()
        df_good = df_good[np.isin(df_good.index.month,seasonz[season])]
        df_good[:5].to_csv(path_to_write+'good_forecasts_{}_lon_{}_{}_lat_{}_{}_{}.txt'.format(column,lon_0,lon_1,
                                                                                               lat_0,lat_1,season))

        #Bad forecasts
        column_MAE = column+'_MAE'
        df_bad = df_MAE_ERA5RF.loc[:,[column]].dropna().rename(columns={column:column_MAE})
        df_bad[column+'_GSS']=df_GSS_ERA5RF.loc[:,[column]].dropna()
        # df_MAE['rank'] = df_MAE[column_MAE].rank(method='min')
        df_bad = df_bad.sort_values(column+'_MAE',ascending=False).where(df_bad[column+'_GSS']<0.2).dropna()
        df_bad = df_bad[np.isin(df_bad.index.month,seasonz[season])]
        df_bad[:5].to_csv(path_to_write+'bad_forecasts_{}_lon_{}_{}_lat_{}_{}_{}.txt'.format(column,lon_0,lon_1,
                                                                                          lat_0,lat_1,season))        

In [9]:
columnz = ['persistent_hw','short_hw','persistent_cw','short_cw']
total_GFS = []; total_ERA5RF = []
for column in columnz:
    column_MAE = column+'_MAE'
    df_MAE = df_MAE_GFS.loc[:,[column]].dropna().rename(columns={column:column_MAE})
    df_MAE['rank'] = df_MAE[column_MAE].rank(method='min')
    df_MAE = df_MAE.sort_values(['rank'])
    column_GSS = column+'_GSS'
    df_GSS = df_GSS_GFS.loc[:,[column]].dropna().rename(columns={column:column_GSS})
    df_GSS['rank'] = df_GSS[column_GSS].rank(method='min',ascending=False)
    df_GSS = df_GSS.sort_values(['rank'])

    df_combined_rank = df_MAE.copy()
    df_combined_rank[column_GSS] = df_GSS[column_GSS]
    df_combined_rank=df_combined_rank.reindex(columns=[column_MAE,column_GSS,'rank'])
    df_combined_rank['rank']=((df_GSS['rank']+df_MAE['rank'])/2)
    df_combined_rank = df_combined_rank.sort_values(['rank'])
    df_combined_rank.to_csv(path+'rank_forecast/forecast_ranking_lead_day_5_{}_GFS.csv'.format(column))
    total_GFS.append(df_combined_rank)
    
    column_MAE = column+'_MAE'
    df_MAE = df_MAE_ERA5RF.loc[:,[column]].dropna().rename(columns={column:column_MAE})
    df_MAE['rank'] = df_MAE[column_MAE].rank(method='min')
    df_MAE = df_MAE.sort_values(['rank'])
    column_GSS = column+'_GSS'
    df_GSS = df_GSS_ERA5RF.loc[:,[column]].dropna().rename(columns={column:column_GSS})
    df_GSS['rank'] = df_GSS[column_GSS].rank(method='min',ascending=False)
    df_GSS = df_GSS.sort_values(['rank'])

    df_combined_rank = df_MAE.copy()
    df_combined_rank[column_GSS] = df_GSS[column_GSS]
    df_combined_rank=df_combined_rank.reindex(columns=[column_MAE,column_GSS,'rank'])
    df_combined_rank['rank']=((df_GSS['rank']+df_MAE['rank'])/2)
    df_combined_rank = df_combined_rank.sort_values(['rank'])
    df_combined_rank.to_csv(path+'rank_forecast/forecast_ranking_lead_day_5_{}_ERA5RF.csv'.format(column))
    total_ERA5RF.append(df_combined_rank)

    
